# Demo - Relative Humidity

This Demo is designed to show the functionality of the Relative Humidity Class 
and the energyCalcs class.

energyCalcs class contains the Vant Hoff acceleration factor and Arrhenius 
Equations Acceleration Factor

To demonstrate we use a processed TMY dataset from "Saudi Arabia, Riyad" that has already calculated the 
Module Temperature using the pvlib library.

There are currently 4 selections for relative Humidity
   
   1) RHsurfaceOutside : Relative Humidity of the Surface of a Solar Module 
   
   2) RHfrontEncapsulant : Relative Humidity of the Frontside Encapsulant of a Solar Module
   
   3) RHbackEncapsulant : Relative Humidity of the backside Encapsulant of a Solar Module 
   
   4) RHbacksheet : Relative 

In [1]:
import pandas as pd
#import Relative_Humidity_for_Solar_Modules
import PVDegradationTools as PVD
import os

path = os.path.join('..','..','PVDegradationTools','data','722024TYA.pickle')
#import data
locationData , processedData_df = pd.read_pickle( path )

#Get the Relative Humidity of outside environment (TMY raw data)
rH_ambient = processedData_df['Relative humidity(%)']

#Get the ambient temperature of outside environment (TMY raw data)
ambient_temp = processedData_df['Dry-bulb temperature(C)']

#Get the temperature of the module (Calulated with pvlib to obtain module temperature)
#We will use open_rack_cell_glassback for this demo
surface_temp = processedData_df['Module Temperature(roof_mount_cell_glassback)(C)']


dni = processedData_df['Direct normal irradiance(W/m^2)']


### parameters

In [2]:
#So : Encapsulant solubility prefactor (g/cm3).  The suggested value for EVA is 1.81390702(g/cm3)
So = 1.81390702
#Eas : Encapsulant solubility activation energy in (kJ/mol).  The suggested value for EVA is 16.729(kJ/mol) 
Eas = 16.729
#Ead : Encapsulant diffusivity activation energy in (kJ/mol) The suggested value for EVA is 38.14(kJ/mol).
Ead = 38.14
#SDW: Diffusivity weighted water content 
SDw = PVD.relativeHumidity.SDw( rH_ambient , ambient_temp , surface_temp, So ,  Eas , Ead)
#WVTRo : Water Vapor Transfer Rate prefactor (g/m2/day). The suggested value for EVA is  7970633554(g/m2/day).
WVTRo = 7970633554
#EaWVTR : Water Vapor Transfer Rate activation energy (kJ/mol) .
# It is suggested to use 0.15(mm) thick PET as a default for the backsheet and set EaWVTR=55.0255(kJ/mol)
EaWVTR = 55.0255
#l : Thickness of the backside encapsulant (mm). The suggested value for encapsulat in EVA,  l=0.5(mm)
l = .5

### Relative Humidity of a Solar Module 

In [3]:
#Get the Relative Humidity of the outside surface of the Solar Module.
RHsurfaceOutside = pd.Series(name="RHsurfaceOutside" , data= \
                      PVD.relativeHumidity.RHsurfaceOutside(rH_ambient, ambient_temp, surface_temp ) )

#Get the Relative Humidity of the Frontside Encapsulant of a Solar Module.
RHfrontEncap = pd.Series(name="RHfront" , data= \
                      PVD.relativeHumidity.RHfront( surface_temp, SDw , So , Eas) )

#Get the Relative Humidity of the Backside Encapsulant of a Solar Module 
RHbackEncap = pd.Series(name="RHbackEncap" , data= \
                      PVD.relativeHumidity.RHbackEncap( rH_ambient , ambient_temp , surface_temp , WVTRo , EaWVTR , So , l , Eas ) )
  
#Get the Relative Humidity of the backside Back sheet of a Solar Module 
RHbacksheet = pd.Series(name="RHbacksheet" , data= \
                      PVD.relativeHumidity.RHbacksheet( RHbackEncap , RHsurfaceOutside ) )

In [4]:
table = {'DNI':dni,
         'RH_ambient':rH_ambient,
         'RHsurfOut':RHsurfaceOutside,
         'RHfrontEncap':RHfrontEncap,
         'RHbackEncap':RHbackEncap,
         'RHbackSheet':RHbacksheet}
demo = pd.DataFrame(table)
demo.iloc[:24]

,DNI,RH_ambient,RHsurfOut,RHfrontEncap,RHbackEncap,RHbackSheet
0,0,88,88.000000,53.091266,472.655534,280.327767
1,0,83,83.000000,51.841704,442.857217,262.928609
2,0,88,88.000000,54.379905,447.889532,267.944766
3,0,94,94.000000,51.841704,410.557230,252.278615
4,0,88,88.000000,54.379905,415.506322,251.753161
5,0,94,94.000000,53.091266,391.101001,242.550501
6,0,94,94.000000,53.091266,377.222042,235.611021
7,0,88,76.274261,48.067200,325.967539,201.120900
8,317,78,35.002265,35.924469,219.791027,127.396646
9,670,65,14.369353,25.488354,120.462950,67.416152


### Vant Hoff Characterization

In [5]:
#PARAMETERS
#Tf = multiplier for the increase in degradation for every 10(C) temperature increase
Tf = 1.41
#x = fit parameter
x = .64

#Temperature equivalent for Vant Hoff Equation
VantHoff_Toeq = PVD.energyCalcs.ToeqVantHoff( Tf, surface_temp )

#IWa : Environment Characterization (W/m^2)
#*for one year of degredation the controlled environmnet lamp settings will 
#    need to be set to IWa
VantHoff_Iwa = PVD.energyCalcs.IwaVantHoff( processedData_df['POA Global(W/m^2)'] ,
                                                         x , 
                                                         Tf , 
                                                         surface_temp ,
                                                         VantHoff_Toeq)

### Vant Hoff Equation Acceleration Factor

In [6]:
#Ichamber = Irradiance of the chamber settings
Ichamber = 1000
#Reference temperature of the chamber (C)
refTemp = 60

#Get the Vant Hoff equation acceleration factor 
VantHoff_AF = PVD.energyCalcs.vantHoffDeg( x , 
            Ichamber , 
            processedData_df['POA Global(W/m^2)'] , 
            surface_temp , 
            Tf , 
            refTemp)

AttributeError: type object 'energyCalcs' has no attribute 'accelerationFactor'

### Arrhenius Characterization

In [ ]:
# Ea = Degredation Activation Energy (kJ/mol)
Ea = 28

#Arrhenius_Teq = Temperature equivalent
Arrhenius_Teq = PVD.energyCalcs.TeqArrhenius( surface_temp , Ea )

# n = fit parameter for relative humidity 
n=1

#RHwa : Relative Humidity Weighted Average
#Use the Relative humidity surface Outside 
Arrhenius_RHwa = PVD.energyCalcs.RHwaArrhenius( RHsurfaceOutside ,
                                            n , 
                                            Ea ,
                                            surface_temp, 
                                            Arrhenius_Teq )

Arrhenius_Iwa = PVD.energyCalcs.IwaArrhenius( processedData_df['POA Global(W/m^2)'],
                                                           x ,
                                                           RHsurfaceOutside ,
                                                           n ,
                                                           surface_temp ,
                                                           Ea ,
                                                           Arrhenius_RHwa,
                                                           Arrhenius_Teq)

### Arrhenius Equation Acceleration Factor


In [ ]:
#rhChamber = Relative Humidity of the controlled environment % "chamber"
rhChamber = 15
#Get the Arrehnius equation acceleration factor
Arrehnius_AF = PVD.energyCalcs.arrheniusCalc( x ,
                                         Ichamber ,
                                         rhChamber ,
                                         n ,
                                         RHsurfaceOutside ,
                                         processedData_df['POA Global(W/m^2)'] ,
                                         refTemp ,
                                         surface_temp,
                                         Ea)